In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from rapidfuzz import process, fuzz

def fill_missing_gvkeys(call_list_path, compustat_path, output_path, fuzzy_threshold=80):
    # Load the datasets
    call_list = pd.read_csv(call_list_path)
    compustat = pd.read_csv(compustat_path)
    
    # Ensure the 'gvkey' column exists in call_list. (If prefilled values exist but are wrong, 
    # we will override them using the company name match.)
    call_list['gvkey'] = np.nan

    # -------------------------------------------------------
    # Preprocessing: Clean and normalize text fields.
    # Convert fields to strings (to avoid errors) then strip and lowercase.
    call_list['nameraw_clean'] = call_list['nameraw'].astype(str).str.strip().str.lower()
    call_list['ticker_clean'] = call_list['ticker'].astype(str).str.strip().str.lower()
    compustat['conm_clean'] = compustat['conm'].astype(str).str.strip().str.lower()
    compustat['tic_clean'] = compustat['tic'].astype(str).str.strip().str.lower()

    print("Initial missing gvkey count:", call_list['gvkey'].isna().sum())

    # -------------------------------------------------------
    # Step 1: Name-based matching (exact match) – give name priority.
    # Deduplicate compustat by company name.
    compustat_by_name = compustat.drop_duplicates(subset=['conm_clean'])
    print("Unique compustat records by name after deduplication:", len(compustat_by_name))
    
    # Merge on the cleaned company name.
    name_merged = pd.merge(
        call_list,
        compustat_by_name[['gvkey', 'conm_clean']],
        left_on='nameraw_clean',
        right_on='conm_clean',
        how='left',
        suffixes=("", "_comp")
    )
    # Replace (or set) the gvkey based on name match.
    call_list['gvkey'] = name_merged['gvkey']
    print("Missing gvkey count after exact name-based merge:", call_list['gvkey'].isna().sum())

    # -------------------------------------------------------
    # Step 2: Ticker-only matching for any records still missing gvkey.
    missing_mask = call_list['gvkey'].isna()
    if missing_mask.any():
        # Deduplicate compustat by ticker.
        compustat_by_ticker = compustat.drop_duplicates(subset=['tic_clean'])
        ticker_merged = pd.merge(
            call_list.loc[missing_mask],
            compustat_by_ticker[['gvkey', 'tic_clean']],
            left_on='ticker_clean',
            right_on='tic_clean',
            how='left',
            suffixes=("", "_comp")
        )
        # Update missing gvkey values with ticker-only match results.
        call_list.loc[missing_mask, 'gvkey'] = ticker_merged['gvkey']
        
    print("Missing gvkey count after ticker-only merge:", call_list['gvkey'].isna().sum())

    # -------------------------------------------------------
    # Step 3: Fuzzy matching on company name for any records still missing gvkey.
    still_missing = call_list[call_list['gvkey'].isna()]
    if not still_missing.empty:
        print("Starting fuzzy matching for {} records".format(len(still_missing)))
        # Create a dictionary for fast lookup: key = cleaned compustat name, value = gvkey.
        compustat_dict = compustat_by_name.set_index('conm_clean')['gvkey'].to_dict()
        compustat_names = list(compustat_dict.keys())
        
        # Loop through each missing record and try to find a fuzzy name match.
        for idx in tqdm(still_missing.index, desc="Fuzzy Matching"):
            candidate_name = call_list.loc[idx, 'nameraw_clean']
            best_match, score, _ = process.extractOne(candidate_name, compustat_names, scorer=fuzz.token_set_ratio)
            if score >= fuzzy_threshold:
                call_list.loc[idx, 'gvkey'] = compustat_dict[best_match]
        print("Missing gvkey count after fuzzy matching:", call_list['gvkey'].isna().sum())

    # -------------------------------------------------------
    # Drop helper columns.
    call_list.drop(columns=['nameraw_clean', 'ticker_clean'], inplace=True)
    
    # Save the updated call list to a new file.
    call_list.to_csv(output_path, index=False)
    print(f"Final missing gvkey count: {call_list['gvkey'].isna().sum()}")
    print(f"Updated call list saved to {output_path}")

# --- File paths (update these paths as needed) ---
call_list_path = "call_list2412.csv"      # Your call_list dataset path
compustat_path = "Compustat2000_2024.csv"   # Your compustat dataset path
output_path = "updated_call_list.csv"       # Desired output file path

# Run the function
fill_missing_gvkeys(call_list_path, compustat_path, output_path)

Initial missing gvkey count: 336844
Unique compustat records by name after deduplication: 31046
Missing gvkey count after exact name-based merge: 336844
Missing gvkey count after ticker-only merge: 336844
Starting fuzzy matching for 336844 records


Fuzzy Matching: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 336844/336844 [4:44:44<00:00, 19.72it/s]


Missing gvkey count after fuzzy matching: 25651
Final missing gvkey count: 25651
Updated call list saved to updated_call_list.csv


In [2]:
new_call_list = pd.read_csv("updated_call_list.csv")

In [3]:
new_call_list

,id,head_year,nameraw,tickerraw,ticker,permno,gvkey
0,595745,2001,Aquila SA,ALAQU.PA,ALAQU,17849,11056.0
1,525773,2001,EMC Corp,EMC^I16,EMC^I16,80491,4510.0
2,118519,2001,Goodrich Corp,GR.N^G12,GR,12140,5229.0
3,601705,2001,Encompass Health Corp,EHC,EHC,10693,12589.0
4,596752,2001,US Dry Cleaning Services Corp,UDRYQ.PK^I11,UDRYQ,25399,29689.0
...,...,...,...,...,...,...,...
336839,15447677,2023,Varroc Engineering Ltd,VARE.NS,VARE,77257,NaN
336840,15444265,2023,Sonos Inc,SONO.OQ,SONO,18011,33767.0
336841,15444012,2023,Scansource Inc,SCSC.OQ,SCSC,80362,29910.0
336842,15608165,2023,UNIQA Insurance Group AG,UNIQ.VI,UNIQ,77814,144496.0


In [4]:
print("missing gvkey count:", new_call_list['gvkey'].isna().sum())

missing gvkey count: 25651


In [5]:
# Filter for rows with missing gvkeys
missing_gvkeys = new_call_list[new_call_list['gvkey'].isna()]

# Select only the company names and tickers, and remove duplicates
missing_names_tickers = missing_gvkeys[['nameraw', 'ticker']].drop_duplicates()

# Print the result
print("Companies with missing gvkeys:")
print(missing_names_tickers.to_string(index=False))

Companies with missing gvkeys:
                                                 nameraw         ticker
                                    2009 Real Estate LLC          WCIMQ
                                             Seal123 Inc          WTSLQ
                               Madison Dearborn Partners          **MDP
                                      Pawnbroker.com Inc           EAGM
                              LBA Realty Fund II WBP LLC          LBARP
                                                 UST LLC            UST
                                         PerkinElmer Inc            PKI
                                      Ingersoll-Rand PLC             IR
                                  Lexington Realty Trust            LXP
                                      Gemalto Holding BV            GTO
                                Bear Stearns (Duplicate)            NaN
                                         CenturyLink Inc            CTL
                                 